In [25]:
import pickle
from tqdm import tqdm
import pandas as pd
import re
import openai
from config import OPENAI_API_KEY
import dask

In [55]:
openai.api_key = OPENAI_API_KEY

In [8]:
data_df = pd.read_csv('wiki_intro_processed.csv')

In [44]:
def chunkify(lst, n):
    return [lst[i:i + n] for i in range(0, len(lst), n)]

In [5]:
def create_prompt(title, starter_text):
    return f'''200 word wikipedia style introduction on '{title}'
    {starter_text}'''

In [33]:
def get_openai_response(dct): 
    title = dct['title']
    starter_text = dct['starter_text']

    prompt = create_prompt(title, starter_text)

    return {title: openai.Completion.create(
        model="text-curie-001",
        prompt=prompt,
        temperature=0.7,
        max_tokens=300,
        top_p=1,
        frequency_penalty=0.4,
        presence_penalty=0.1
    )}

In [35]:
def fake_get_openai_response(dct):
    title = dct['title']
    starter_text = dct['starter_text']

    prompt = create_prompt(title, starter_text)

    return {title: f'{prompt} blah blah blah'}

In [54]:
def run_dask(lst_dct):
    delayed_calls = [dask.delayed(get_openai_response)(dct) for dct in lst_dct]
    results = dask.compute(*delayed_calls)
    return results

In [70]:
chunk_n = 0
for chunk in chunkify(data_df[['title', 'starter_text']].to_dict('records'), 1000):
    if chunk_n == 0:
        chunk_n += 1
        continue
    print(chunk_n)
    result = run_dask(chunk)
    with open(f'data/result-{chunk_n}.pkl', 'wb') as file:
        pickle.dump(result, file)
    chunk_n += 1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
